<center>
<h4>CDS 110, Lecture 2</h4>
<font color=blue><h1>Nonlinear Dynamics (and Control) of an Inverted Pendulum System</h1></font>
<h3>Richard M. Murray, Winter 2024</h3>
</center>

[Open in Google Colab](https://colab.research.google.com/drive/1is083NiFdHcHX8Hq56oh_AO35nQGO4bh)

In this lecture we investigate the nonlinear dynamics of an inverted pendulum system.  More information on this example can be found in [FBS2e](https://fbswiki.org/wiki/index.php?title=FBS), Examples 3.3 and 5.4.  This lecture demonstrates how to use [python-control](https://python-control.org) to analyze nonlinear systems, including creating phase plane plots.


In [ ]:
# Import the packages needed for the examples included in this notebook
import numpy as np
import matplotlib.pyplot as plt
from math import pi
try:
  import control as ct
  print("python-control", ct.__version__)
except ImportError:
  !pip install control
  import control as ct

## System model

We consider an invereted pendulum, which is a simplified version of a balance system:

<center><img src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/invpend-diagram.png" alt="invpend.diagram" width=100></center>

The dynamics for an inverted pendulum system can be written as:

$$
  \dfrac{d}{dt} \begin{bmatrix} \theta \\ \dot\theta\end{bmatrix} =
    \begin{bmatrix}
      \dot\theta \\
        \dfrac{m g l}{J_\text{t}} \sin \theta
      - \dfrac{b}{J_\text{t}} \dot\theta
      + \dfrac{l}{J_\text{t}} u \cos\theta
    \end{bmatrix}, \qquad
    y = \theta,
$$

where $m$ and $J_t = J + m l^2$ are the mass and (total) moment of inertia of the system to be balanced, $l$ is the distance from the base to the center of mass of the balanced body, $b$ is the coefficient of rotational friction, and $g$ is the acceleration due to gravity.

We begin by creating a nonlinear model of the system:

In [ ]:
invpend_params = {'m': 1, 'l': 1, 'b': 0.5, 'g': 1}
def invpend_update(t, x, u, params):
    m, l, b, g = params['m'], params['l'], params['b'], params['g']
    umax = params.get('umax', 1)
    usat = np.clip(u[0], -umax, umax)
    return [x[1], -b/m * x[1] + (g * l / m) * np.sin(x[0] + usat/m)]
invpend = ct.nlsys(
    invpend_update, states=['theta', 'thdot'],
    inputs=['tau'], outputs=['theta', 'thdot'],
    params=invpend_params, name='invpend')
print(invpend)

## Open loop dynamics

The open loop dynamics of the system can be visualized using the `phase_plane_plot` command in python-control:

In [ ]:
ct.phase_plane_plot(
    invpend, [-2*pi - 1, 2*pi + 1, -2, 2], 8),

# Draw lines at the downward equilibrium angles
plt.plot([-pi, -pi], [-2, 2], 'k--')
plt.plot([pi, pi], [-2, 2], 'k--')

We see that the vertical ($\theta = 0$) equilibrium point is unstable, but the downward equlibrium points ($\theta = \pm \pi$) are stable.

Note also the *separatrices* for the equilibrium point, which gives insights into the regions of attraction (the red dashed line separates the two regions of attraction).

## Proportional feedback

We now stabilize the system using a simple proportional feedback controller:

$$u = -k_\text{p} \theta.$$

This controller can be designed as an input/output system that has no state dynamics, just a mapping from the inputs to the outputs:

In [ ]:
# Set up the controller
def propctrl_output(t, x, u, params):
  kp = params.get('kp', 1)
  return -kp * (u[0] - u[1])
propctrl = ct.nlsys(
    None, propctrl_output, name="p_ctrl",
    inputs=['theta', 'r'], outputs='tau'
)
print(propctrl)

Note that the input to the controller is the reference value $r$ (which we will always take to be zero), the measured output $y$, which is the angle $\theta$ for our system.  The output of the controller is the system input $u$, corresponding to the force applied to the wheels.

To connect the controller to the system, we use the [`interconnect`](https://python-control.readthedocs.io/en/latest/generated/control.interconnect.html) function, which will connect all signals that have the same names:

In [ ]:
# Create the closed loop system
clsys = ct.interconnect(
    [invpend, propctrl], name='invpend w/ proportional feedback',
    inputs=['r'], outputs=['theta', 'tau'], params={'kp': 1})
print(clsys)

Note: you will see a warning when you run this command, because the output $\dot\theta$ (`thdot`) is not connected to anything.  You can ignore this here, but as you get to more complicated examples, you should pay attention to warnings of this sort and make sure they are OK.

We can now linearize the closed loop system at different gains and compute the eigenvalues to check for stability:

In [ ]:
# Solution
for kp in [0, 1, 10]:
  print("kp = ", kp, "; poles = ", clsys.linearize([0, 0], [0], params={'kp': kp}).poles())

We see that at $k_\text{p} = 10$ the eigenvalues (poles) of the closed loop system both have negative real part, and so the system is stabilized.

### Phase portrait

To study the resulting dynamics, we try plotting a phase plot using the same commands as before, but now for the closed loop system (with appropriate proportional gain):

In [ ]:
ct.phase_plane_plot(
    clsys, [-2*pi, 2*pi, -2, 2], 8, params={'kp': 10});

This plot is not very useful and has several errors.  It shows the limitations of the default parameter values for the `phase_plane_plot` command.

Some things to notice in this plot:
* Not all of the equilibrium points are showing up (there are two unstable equilibrium points that are missing)
* There is no detail about what is happening near the origin.

### Improved phase portrait

To fix these issues, we can do a couple of things:
* Restrict the range of the plot from $-3\pi/2$ to $3\pi/2$, which means that grid used to calculate the equilibrium point is a bit finer.
* Reset the grid spacing, so that we have more initial conditions around the edge of the plot and a finer search for equilibrium points.

Here's some improved code:

In [ ]:
kp_params = {'kp': 10}
ct.phase_plane_plot(
    clsys, [-1.5 * pi, 1.5 * pi, -2, 2], 8,
    gridspec=[13, 7], params=kp_params,
    plot_separatrices={'timedata': 5})
plt.plot([-pi, -pi], [-2, 2], 'k--', [ pi,  pi], [-2, 2], 'k--')
plt.plot([-pi/2, -pi/2], [-2, 2], 'k:', [ pi/2,  pi/2], [-2, 2], 'k:');

In [ ]:
# Play around with some paramters to see what happens
fig, axs = plt.subplots(2, 2)
for i, kp in enumerate([3, 10]):
  for j, umax in enumerate([0.2, 1]):
    ct.phase_plane_plot(
      clsys, [-1.5 * pi, 1.5 * pi, -2, 2], 8,
      gridspec=[13, 7], plot_separatrices={'timedata': 5},
      params={'kp': kp, 'umax': umax}, ax=axs[i, j])
    axs[i, j].set_title(f"{kp=}, {umax=}")
plt.tight_layout()

## State space controller

For the proportional controller, we have limited control over the dynamics of the closed loop system.  For example, we see that the solutions near the origin are highly oscillatory in both the $k_\text{p} = 3$ and $k_\text{p} = 10$ cases.

An alternative is to use "full state feedback", in which we set

$$
u = -K (x - x_\text{d}) = -k_1 (\theta - \theta_d) - k_2 (\dot\theta - \dot\theta_d).
$$

We will learn more about how to design these controllers later, so if you aren't familiar with the idea of eigenvalue placement, just take this as a bit of "control theory magic" for now.

To compute the gains, we make use of the `place` command, applied to the linearized system:

In [ ]:
# Linearize the system
P = invpend.linearize([0, 0], [0])

# Place the closed loop eigenvalues (poles) at desired locations
K = ct.place(P.A, P.B, [-1 + 0.1j, -1 - 0.1j])
print(f"{K=}")

In [ ]:
def statefbk_output(t, x, u, params):
  K = params.get('K', np.array([0, 0]))
  return -K @ (u[0:2] - u[2:])
statefbk = ct.nlsys(
    None, statefbk_output, name="k_ctrl",
    inputs=['theta', 'thdot', 'theta_d', 'thdot_d'], outputs='tau'
)
print(statefbk)

In [ ]:
clsys_sf = ct.interconnect(
    [invpend, statefbk], name='invpend w/ state feedback',
    inputs=['theta_d', 'thdot_d'], outputs=['theta', 'tau'], params={'kp': 1})
print(clsys_sf)

### Phase portrait

In [ ]:
ct.phase_plane_plot(
    clsys_sf, [-1.5 * pi, 1.5 * pi, -2, 2], 8,
    gridspec=[13, 7], params={'K': K})
plt.plot([-pi, -pi], [-2, 2], 'k--', [ pi,  pi], [-2, 2], 'k--')
plt.plot([-pi/2, -pi/2], [-2, 2], 'k:', [ pi/2,  pi/2], [-2, 2], 'k:')

Note that the closed loop response around the upright equilibrium point is much less oscillatory (consistent with where we placed the closed loop eigenvalues of the system dynamics).

## Things to try

Here are some things to try with the above code:
* Try changing the locations of the closed loop eigenvalues in the `place` command
* Try resetting the limits of the control action (`umax`)
* Try leaving the state space controller fixed but changing the parameters of the system dynamics ($m$, $l$, $b$).  Does the controller still stabilize the system?
* Plot the initial condition response of the system and see how to map time traces to phase plot traces.